# 实验一
自定义Inception网络，并在Kaggle猫/狗数据集上进行训练和测试

## 1.加载keras模块

In [7]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Conv2D, AveragePooling2D,Input,BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense,Concatenate
from keras.utils import to_categorical
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras import backend as K
import numpy as np

### 定义Inception网络结构



In [8]:
img_width, img_height = 50, 50
#image_data_format是“返回默认图像的维度顺序”
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
    bn_axis = 1
else:
    input_shape = (img_width, img_height, 3)
    bn_axis = 3

x = Input (input_shape)    
#x = AveragePooling2D(
#        pool_size=(3, 3),strides=(1, 1), data_format=K.image_data_format())(x)

#如果padding设置为same，则说明输入图片大小和输出图片大小是一致的，如果是valid则图片经过滤波器后可能会变小
#branch 1
branch1_out = Conv2D(16, (1, 1),padding="same",use_bias=False)(x)   
#bias是“偏置”      
#卷积之后，如果要接BN操作，最好是不设置偏置，因为不起作用，而且占显卡内存。
branch1_out = BatchNormalization(axis=bn_axis)(branch1_out)
branch1_out = Activation('relu')(branch1_out)
#BatchNormalization是“批量规范化 ”   
    
#branch 2   
branch2_out = Conv2D(16, (1, 1),padding="same",use_bias=False)(x)
branch2_out = BatchNormalization(axis=bn_axis)(branch2_out)
branch2_out = Activation('relu')(branch2_out)

branch2_out = Conv2D(48, (3, 3),padding="same",use_bias=False)(branch2_out)
branch2_out = BatchNormalization(axis=bn_axis)(branch2_out)
branch2_out = Activation('relu')(branch2_out) 


#branch 3
branch3_out = Conv2D(16, (1, 1),padding="same",use_bias=False)(x)
branch3_out = BatchNormalization(axis=bn_axis)(branch3_out)
branch3_out = Activation('relu')(branch3_out)

branch3_out = Conv2D(24, (5, 5),padding="same",use_bias=False)(branch3_out)
branch3_out = BatchNormalization(axis=bn_axis)(branch3_out)
branch3_out = Activation('relu')(branch3_out) 

#branch 4
branch4_out = AveragePooling2D(pool_size=(3, 3),strides=(1, 1), padding='same', data_format=K.image_data_format())(x)

branch4_out = Conv2D(16, (1, 1),padding="same",use_bias=False)(branch4_out)
branch4_out = BatchNormalization(axis=bn_axis)(branch4_out)
branch4_out = Activation('relu')(branch4_out) 

#concatenate layer（连接层）
out = Concatenate(axis=bn_axis)([branch1_out, branch2_out, branch3_out, branch4_out])
out = Conv2D(16, (1, 1),padding="same",use_bias=False)(out)

#fully connected layer（全连接层）
out = Flatten()(out)
out = Dense(48, activation='relu')(out)
#output layer
out = Dense(1, activation='sigmoid')(out)

model = Model(x, out)

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

### 查看model架构



In [9]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 50, 50, 3)    0                                            
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 50, 50, 16)   48          input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 50, 50, 16)   48          input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_8 (BatchNor (None, 50, 50, 16)   64          conv2d_11[0][0]                  
____________________________________________________________________________________________

### 定义ImageDataGenerator


In [4]:
train_data_dir = r'C:\Users\Administrator\Desktop\kejian\an11.6\dogs-vs-cats\train'
validation_data_dir = r'C:\Users\Administrator\Desktop\keji\11.6\dogs-vs-cats\validation'
nb_train_samples = 10835
nb_validation_samples = 4000
epochs = 1
batch_size = 5

#ImageDataGenerator是“图像数据生成器”
# this is the augmentation configuration we will use for training（这是我们将用于训练的增强配置）
train_datagen = ImageDataGenerator(
    rescale=1. / 255,      #重新缩放
    shear_range=0.2,      #剪切范围
    zoom_range=0.2,     #缩放范围
    horizontal_flip=True)  #水平翻转

# this is the augmentation configuration we will use for testing:（这是我们将用于测试的增强配置：）
# only rescaling（仅重新缩放）
test_datagen = ImageDataGenerator(rescale=1. / 255)



train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')
#validation验证   target目标   batch批量  generator发生器
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')



Found 10835 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


### 训练模型



In [5]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/1
2167/2167 [==============================] - 194s 90ms/step - loss: 0.8479 - accuracy: 0.5710 - val_loss: 0.7436 - val_accuracy: 0.5893


### 使用训练后模型预测图像





In [11]:
import cv2

img = cv2.resize(cv2.imread(r'C:\Users\Administrator\Desktop\kejian\11.6\dogs-vs-cats\test\7.jpg'), (img_width, img_height)).astype(np.float32)
# img[:,:,0] -= 103.939
# img[:,:,1] -= 116.779
# img[:,:,2] -= 123.68
#img = img.transpose((2,0,1))
x = img_to_array(img)

x = np.expand_dims(x, axis=0)

#x = preprocess_input(x)

score = model.predict(x)


print(score)

[[1.]]
